In [1]:
import os

def get_files_in_subdirectories(directory):
    file_list = []
    for root, _, files in os.walk(directory):
        for file in files:
            file_list.append(os.path.join(root, file))
    return file_list

# Thay đổi đường dẫn tới thư mục của bạn ở đây
folder_path = './crawled'
files = get_files_in_subdirectories(folder_path)
print(len(files))


11


In [2]:
files

['./crawled\\danang.txt',
 './crawled\\diendandulich.txt',
 './crawled\\full_nhandan.txt',
 './crawled\\full_nhandanthoinay.txt',
 './crawled\\vinpearl.jsonl',
 './crawled\\csdl_dulich\\co_so_luu_tru.csv',
 './crawled\\csdl_dulich\\diem_den_du_lich.txt',
 './crawled\\csdl_dulich\\the_thao.txt',
 './crawled\\dulich3mien\\all_content_dulich3mien.txt',
 './crawled\\dulich3mien\\all_title_dulich3mien.txt',
 './crawled\\dulich3mien\\cauhoithuonggap.txt']

In [3]:
import fileinput
def replace_string_in_files(directory, old_string, new_string):
    for root, directories, files in os.walk(directory):
        for file_name in files:
            file_path = os.path.join(root, file_name)
            # Đảm bảo tệp đang xử lý là tệp văn bản
            if not file_path.endswith(('.txt', '.csv', '.py', '.html', '.css')):
                continue
            with fileinput.FileInput(file_path, inplace=True) as file:
                for line in file:
                    print(line.replace(old_string, new_string), end='')

In [4]:
old_string = '=================================================='
new_string = '\n'
replace_string_in_files(folder_path, old_string, new_string)

In [35]:
import os
import csv
import json
from tqdm import tqdm


def write_csv_records_to_text(csv_file):
    filename, _ = os.path.splitext(csv_file)

    with open(csv_file, 'r', newline='', encoding='utf-8') as csvfile, \
         open(f"{filename}.txt", 'w', encoding='utf-8') as txtfile:
        
        csv_reader = csv.DictReader(csvfile)
        fieldnames = csv_reader.fieldnames

        for row in csv_reader:
            for field in fieldnames:
                txtfile.write(f"{field.capitalize()}: {row[field]}\n")
            txtfile.write("\n\n")
    os.remove(csv_file)
    return  os.path.basename(f"{filename}.txt")



def write_jsonl_to_text(jsonl_file):
    filename, _ = os.path.splitext(jsonl_file)

    with open(jsonl_file, 'r', encoding='utf-8') as jsonlfile, \
         open(f"{filename}.txt", 'w', encoding='utf-8') as txtfile:
        
        for line in jsonlfile:
            data = json.loads(line)
            for key, value in data.items():
                txtfile.write(f"{key.capitalize()}: {value}\n")
            txtfile.write("\n\n")
    os.remove(jsonl_file)
    return f"{filename}.txt"

def clean_file_content(input_file):
    with open(input_file, 'r', encoding='utf-8') as infile:
        lines = infile.readlines()

    with open(input_file, 'w', encoding='utf-8') as outfile:
        for line in lines:
            cleaned_line = line.replace("[", "").replace("]", "").replace("{", "").replace("}", "")
            outfile.write(cleaned_line)
    input_file= os.path.basename(input_file)
    return input_file

def split_text_into_chunks(text, chunk_size=584, window_size=50):
    """Split a long text into multiple chunks (passages) with manageable sizes.
    
    Args:
        chunk_size (int): Maximum size of a chunk.
        window_size (int): Decide how many words are overlapped between two consecutive chunks. Basically #overlapped_words = chunk_size - window_size.
    Returns:
        str: Multiple chunks of text splitted from initial document text.
    """
    words = text.split()
    num_words = len(words)
    chunks = []
    start_idx = 0

    while True:
        end_idx = start_idx + chunk_size
        chunk = " ".join(words[start_idx:end_idx])
        chunks.append(chunk)
        if end_idx >= num_words:
            break
        start_idx += window_size

    return chunks

def get_corpus(data_dir="data/data_raw10k/"):
    """Transform a corpus of documents into a corpus of passages.
    
    Args:
        data_dir (str): directory that contains .txt files, each file contains text content of a Wikipedia page.
    Returns:
        str: A corpus of chunks splitted from multiple initial documents. Each chunk will contain information about (id, title, passage)
    """
    corpus = []
    meta_corpus = []

    for root, _, files in os.walk(data_dir):
        for filename in tqdm(files):
            filepath = os.path.join(root, filename)
            if filename.endswith(".csv"):
                filename = write_csv_records_to_text(filepath)
            elif filename.endswith(".jsonl") or filename.endswith(".json"):
                filename = clean_file_content(write_jsonl_to_text(filepath))
            elif filename.endswith(".txt"):
                title = os.path.splitext(filename)[0]
                with open(filepath, "r") as f:
                    text = f.read().strip()

                chunks = split_text_into_chunks(text, chunk_size=584, window_size=150)
                chunks = [f"Title: {title}\n\n{chunk}" for chunk in chunks]
                meta_chunks = [{
                    "title": title,
                    "passage": chunk,
                    "id": i,
                    "len": len(chunk.split())
                } for i, chunk in enumerate(chunks)]

                corpus.extend(chunks)
                meta_corpus.extend(meta_chunks)

    return meta_corpus

def dump_corpus(meta_corpus):
    file_path = "data/corpus_chunks.jsonl"
    os.makedirs(os.path.dirname(file_path), exist_ok=True)

    with open(file_path, "a", encoding="utf-8") as outfile:
        for chunk in meta_corpus:
            d = json.dumps(chunk, ensure_ascii=False) + "\n"
            outfile.write(d)


if __name__ == "__main__": 
    data_dir = "./crawled/"
    meta_corpus = get_corpus(data_dir)
    dump_corpus(meta_corpus)
    print(meta_corpus[2000])


100%|██████████| 3/3 [00:00<00:00,  3.58it/s]


{'title': 'diem_den_du_lich', 'passage': 'Title: diem_den_du_lich\n\nchiến dịch liên lạc với các đơn vị bộ binh, công binh, pháo binh, cao xạ ở phía trước và các đơn vị kho, trạm của Tổng cục cung cấp, hệ thống quân y, dân công hỏa tuyến ở phía sau mặt trận. Đây là mạng thông tin liên lạc trực tiếp giữa Bộ Chỉ huy chiến dịch với Bộ Chính trị Trung ương Đảng, Bác Hồ.. Từ căn hầm xuyên núi, thông từ lán của Đại tướng Võ Nguyên Giáp đến lán của Thiếu tướng Hoàng Văn Thái, cho tới các điểm khác như nơi làm việc của đoàn cố vấn quân sự Trung Quốc, lán làm việc của Trưởng ban thông tin chiến dịch Hoàng Đạo Thúy… tất cả như vẫn còn như nguyên vẹn dấu ấn của lịch sử... Cạnh nơi làm việc và nghỉ ngơi của Đại tướng là hầm trú ẩn được đào xuyên qua lòng núi. Những lúc quân Pháp ném bom dữ dội, Đại tướng làm việc và nghỉ ngơi trong hầm trú ẩn này. Từ lán Đại tướng Võ Nguyên Giáp thông sang lán Tham mưu trưởng Hoàng Văn Thái và lán cố vấn quân sự Vi Quốc Thanh là một đường hầm dài 69m. Đường hầm ca

In [5]:
from tqdm import tqdm
from googletrans import Translator
import os
import time

def translate_to_en(text):
    translator = Translator()
    translation = translator.translate(text, src='vi', dest='en')
    return translation.text

def translate_files_in_directory(data_dir, new_dir):
    for root, _, files in os.walk(data_dir):
        for filename in tqdm(files):
            input_file_path = os.path.join(root, filename)
            output_file_path = os.path.join(new_dir, f"en_{filename}")

            with open(input_file_path, 'r', encoding='utf-8') as input_file:
                vietnamese_sentences = input_file.readlines()

            with open(output_file_path, 'w', encoding='utf-8') as output_file:
                for i, sentence in enumerate(vietnamese_sentences):
                    if i % 50 == 0 and i != 0:
                        time.sleep(120)
                    sentence = sentence.strip()
                    if sentence: 
                        vietnamese_translation = translate_to_en(sentence)
                        print(vietnamese_translation)
                        output_file.write(vietnamese_translation + '\n')

if __name__ == "__main__":
    data_dir = "E:\\AI_Project\\travel_data_crawler\\crawled"  # Update this with the directory path containing your files
    translate_files_in_directory(data_dir, "./en_data")
    print("Translation completed.")


AttributeError: module 'httpcore' has no attribute 'SyncHTTPTransport'